<a href="https://colab.research.google.com/github/Madushani-Weerasekara/D10_Fake-News-Classifier-Using-LSTM/blob/main/D10_Fake_News_Classifier_Using_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Fake News Classifier Using LSTM

In [1]:
import pandas as pd


In [2]:
df = pd.read_csv('/content/FakeNewsNet.csv')

In [3]:
df.head()

,title,news_url,source_domain,tweet_num,real
0,Kandi Burruss Explodes Over Rape Accusation on...,http://toofab.com/2017/05/08/real-housewives-a...,toofab.com,42,1
1,People's Choice Awards 2018: The best red carp...,https://www.today.com/style/see-people-s-choic...,www.today.com,0,1
2,Sophia Bush Sends Sweet Birthday Message to 'O...,https://www.etonline.com/news/220806_sophia_bu...,www.etonline.com,63,1
3,Colombian singer Maluma sparks rumours of inap...,https://www.dailymail.co.uk/news/article-33655...,www.dailymail.co.uk,20,1
4,Gossip Girl 10 Years Later: How Upper East Sid...,https://www.zerchoo.com/entertainment/gossip-g...,www.zerchoo.com,38,1


In [4]:
df.shape

(23196, 5)

In [5]:
df.isnull().sum()

title              0
news_url         330
source_domain    330
tweet_num          0
real               0
dtype: int64

In [6]:
##Drop Null values
df.dropna()

,title,news_url,source_domain,tweet_num,real
0,Kandi Burruss Explodes Over Rape Accusation on...,http://toofab.com/2017/05/08/real-housewives-a...,toofab.com,42,1
1,People's Choice Awards 2018: The best red carp...,https://www.today.com/style/see-people-s-choic...,www.today.com,0,1
2,Sophia Bush Sends Sweet Birthday Message to 'O...,https://www.etonline.com/news/220806_sophia_bu...,www.etonline.com,63,1
3,Colombian singer Maluma sparks rumours of inap...,https://www.dailymail.co.uk/news/article-33655...,www.dailymail.co.uk,20,1
4,Gossip Girl 10 Years Later: How Upper East Sid...,https://www.zerchoo.com/entertainment/gossip-g...,www.zerchoo.com,38,1
...,...,...,...,...,...
23191,Pippa Middleton wedding: In case you missed it...,https://www.express.co.uk/news/royal/807049/pi...,www.express.co.uk,52,1
23192,Zayn Malik & Gigi Hadid’s Shocking Split: Why ...,hollywoodlife.com/2018/03/13/zayn-malik-gigi-h...,hollywoodlife.com,7,0
23193,Jessica Chastain Recalls the Moment Her Mother...,http://www.justjared.com/2018/01/17/jessica-ch...,www.justjared.com,26,1
23194,"Tristan Thompson Feels ""Dumped"" After Khloé Ka...",www.intouchweekly.com/posts/tristan-thompson-f...,www.intouchweekly.com,24,0


In [7]:
# Get the independent feature
x = df.drop('real', axis=1)

In [8]:
#Get the dependent feature
y=df['real']

In [9]:
x.shape

(23196, 4)

In [10]:
y.shape

(23196,)

In [11]:
!pip install tensorflow

In [12]:
import tensorflow as tf

In [13]:
tf.__version__

'2.15.0'

In [14]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense


In [15]:
##Vocabulary size
voc_size = 5000

##Onehot Representation

In [16]:
messages = x.copy()

In [17]:
# title of the 1st index
messages['title'][1]

"People's Choice Awards 2018: The best red carpet looks"

In [18]:
messages

,title,news_url,source_domain,tweet_num
0,Kandi Burruss Explodes Over Rape Accusation on...,http://toofab.com/2017/05/08/real-housewives-a...,toofab.com,42
1,People's Choice Awards 2018: The best red carp...,https://www.today.com/style/see-people-s-choic...,www.today.com,0
2,Sophia Bush Sends Sweet Birthday Message to 'O...,https://www.etonline.com/news/220806_sophia_bu...,www.etonline.com,63
3,Colombian singer Maluma sparks rumours of inap...,https://www.dailymail.co.uk/news/article-33655...,www.dailymail.co.uk,20
4,Gossip Girl 10 Years Later: How Upper East Sid...,https://www.zerchoo.com/entertainment/gossip-g...,www.zerchoo.com,38
...,...,...,...,...
23191,Pippa Middleton wedding: In case you missed it...,https://www.express.co.uk/news/royal/807049/pi...,www.express.co.uk,52
23192,Zayn Malik & Gigi Hadid’s Shocking Split: Why ...,hollywoodlife.com/2018/03/13/zayn-malik-gigi-h...,hollywoodlife.com,7
23193,Jessica Chastain Recalls the Moment Her Mother...,http://www.justjared.com/2018/01/17/jessica-ch...,www.justjared.com,26
23194,"Tristan Thompson Feels ""Dumped"" After Khloé Ka...",www.intouchweekly.com/posts/tristan-thompson-f...,www.intouchweekly.com,24


In [19]:
messages.reset_index(inplace=True)

In [20]:
messages

,index,title,news_url,source_domain,tweet_num
0,0,Kandi Burruss Explodes Over Rape Accusation on...,http://toofab.com/2017/05/08/real-housewives-a...,toofab.com,42
1,1,People's Choice Awards 2018: The best red carp...,https://www.today.com/style/see-people-s-choic...,www.today.com,0
2,2,Sophia Bush Sends Sweet Birthday Message to 'O...,https://www.etonline.com/news/220806_sophia_bu...,www.etonline.com,63
3,3,Colombian singer Maluma sparks rumours of inap...,https://www.dailymail.co.uk/news/article-33655...,www.dailymail.co.uk,20
4,4,Gossip Girl 10 Years Later: How Upper East Sid...,https://www.zerchoo.com/entertainment/gossip-g...,www.zerchoo.com,38
...,...,...,...,...,...
23191,23191,Pippa Middleton wedding: In case you missed it...,https://www.express.co.uk/news/royal/807049/pi...,www.express.co.uk,52
23192,23192,Zayn Malik & Gigi Hadid’s Shocking Split: Why ...,hollywoodlife.com/2018/03/13/zayn-malik-gigi-h...,hollywoodlife.com,7
23193,23193,Jessica Chastain Recalls the Moment Her Mother...,http://www.justjared.com/2018/01/17/jessica-ch...,www.justjared.com,26
23194,23194,"Tristan Thompson Feels ""Dumped"" After Khloé Ka...",www.intouchweekly.com/posts/tristan-thompson-f...,www.intouchweekly.com,24


In [21]:
messages.shape

(23196, 5)

In [22]:
import nltk
import re
from nltk.corpus import stopwords

In [23]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [24]:
#Dataset Preprocessing
from nltk.stem import PorterStemmer # Stemming purpose
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
  review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
  review = review.lower()
  review = review.split()

  review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
  review = ' '.join(review)
  corpus.append(review)

In [25]:
corpus

['kandi burruss explod rape accus real housew atlanta reunion video',
 'peopl choic award best red carpet look',
 'sophia bush send sweet birthday messag one tree hill co star hilari burton breyton eva',
 'colombian singer maluma spark rumour inappropri relationship aunt',
 'gossip girl year later upper east sider shock world chang pop cultur forev',
 'gwen stefani got dump blake shelton jealousi drama exclus',
 'broward counti sheriff fire lie parkland',
 'amber rose shut french montana date rumor call rapper bruvaaa',
 'mindi kale make first post babi appear disneyland wrinkl time co star',
 'katharin mcphee butcher toni nomin drink',
 'wag miami star ashley nicol robert philip wheeler marri',
 'mel gibson hollywood pedophil nowher left hide',
 'medium tyler henri address chill messag kristin cavallari deceas brother express read',
 'dwt season result week disney night',
 'via cq subscrib tax foundat',
 'reason tarek el moussa overcom latest back injuri',
 'david cassidi cut estrang 

In [26]:
corpus[0]

'kandi burruss explod rape accus real housew atlanta reunion video'

In [27]:
onehot_rep=[one_hot(words,voc_size)for words in corpus]
onehot_rep

[[3480, 2134, 658, 4638, 12, 993, 2460, 1313, 4023, 3470],
 [4643, 3151, 3565, 4953, 1605, 3114, 1822],
 [2699,
  388,
  1406,
  2361,
  1412,
  4916,
  4097,
  1996,
  1621,
  4475,
  3547,
  4485,
  3807,
  1634,
  4760],
 [4248, 4538, 3442, 2427, 2432, 2106, 1798, 945],
 [634, 4138, 195, 4152, 1174, 2031, 4822, 2014, 4613, 1458, 1284, 509, 4639],
 [1923, 1009, 160, 1834, 2215, 3248, 767, 800, 3109],
 [319, 4856, 4933, 540, 288, 998],
 [1110, 2423, 3887, 1102, 2372, 2586, 4247, 2796, 3761, 3732],
 [2789, 3045, 2026, 3067, 807, 3818, 743, 1137, 2703, 1743, 4475, 3547],
 [2000, 2941, 1242, 3618, 4438, 2902],
 [4213, 2700, 3547, 2972, 682, 3903, 3283, 785, 995],
 [1989, 3605, 2249, 3388, 3467, 1919, 311],
 [2587, 2024, 511, 4715, 970, 4916, 4204, 2986, 2109, 4626, 4187, 2410],
 [2321, 4583, 4879, 2512, 3757, 1261],
 [941, 3639, 3234, 233, 3964],
 [1239, 2404, 4949, 106, 2993, 3606, 2520, 2961],
 [4819, 2330, 2351, 3779, 1115, 1983, 1367, 3094, 2105, 1019],
 [1677, 3906, 1758, 481, 2934,

In [28]:
onehot_rep[0]

[3480, 2134, 658, 4638, 12, 993, 2460, 1313, 4023, 3470]

##Embedding Representation

In [29]:
sent_length = 20 #Max length of he sentences
embedded_docs = pad_sequences(onehot_rep, padding = 'pre', maxlen = sent_length)
print(embedded_docs)

[[   0    0    0 ... 1313 4023 3470]
 [   0    0    0 ... 1605 3114 1822]
 [   0    0    0 ... 3807 1634 4760]
 ...
 [   0    0    0 ... 4400  205  668]
 [   0    0    0 ... 1324  955 3109]
 [   0    0    0 ... 1987 1578 3565]]


In [30]:
print(embedded_docs[0])

[   0    0    0    0    0    0    0    0    0    0 3480 2134  658 4638
   12  993 2460 1313 4023 3470]


In [31]:
print(embedded_docs[1])

[   0    0    0    0    0    0    0    0    0    0    0    0    0 4643
 3151 3565 4953 1605 3114 1822]


In [32]:
#Creating model
embedding_vector_features = 40 #Features representation
model = Sequential()
model.add(Embedding(voc_size, embedding_vector_features, input_length = sent_length)) #Add the embedding layer
model.add(LSTM(100)) #Add LSTM with 100 nuerons
model.add(Dense(1, activation = 'sigmoid'))
model.compile(loss ='binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 40)            200000    
                                                                 
 lstm (LSTM)                 (None, 100)               56400     
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 256501 (1001.96 KB)
Trainable params: 256501 (1001.96 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [33]:
len(embedded_docs), y.shape

(23196, (23196,))

In [34]:
import numpy as np
x_final = np.array(embedded_docs)
y_final = np.array(y)

In [35]:
x_final.shape, y_final.shape


((23196, 20), (23196,))

In [36]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x_final, y_final, test_size=0.33, random_state=42 )


##Model Training

In [37]:
model.fit(x_train, y_train, validation_data=(x_test, y_test),epochs = 10, batch_size= 64)

Epoch 1/10
243/243 [==============================] - 12s 39ms/step - loss: 0.4644 - accuracy: 0.7942 - val_loss: 0.4131 - val_accuracy: 0.8240
Epoch 2/10
243/243 [==============================] - 8s 32ms/step - loss: 0.3424 - accuracy: 0.8516 - val_loss: 0.4203 - val_accuracy: 0.8264
Epoch 3/10
243/243 [==============================] - 10s 41ms/step - loss: 0.3053 - accuracy: 0.8678 - val_loss: 0.4411 - val_accuracy: 0.8199
Epoch 4/10
243/243 [==============================] - 9s 38ms/step - loss: 0.2759 - accuracy: 0.8841 - val_loss: 0.4560 - val_accuracy: 0.8125
Epoch 5/10
243/243 [==============================] - 9s 39ms/step - loss: 0.2455 - accuracy: 0.8993 - val_loss: 0.4886 - val_accuracy: 0.8067
Epoch 6/10
243/243 [==============================] - 9s 38ms/step - loss: 0.2188 - accuracy: 0.9101 - val_loss: 0.5462 - val_accuracy: 0.8035
Epoch 7/10
243/243 [==============================] - 10s 41ms/step - loss: 0.1974 - accuracy: 0.9200 - val_loss: 0.5904 - val_accuracy: 0.7

In [43]:
y_pred = (model.predict(x_test)>0.5).astype('int32')

240/240 [==============================] - 1s 6ms/step


In [44]:
from sklearn.metrics import confusion_matrix,accuracy_score
accuracy_score(y_test, y_pred)

0.7942521227955585

In [45]:
print(accuracy_score(y_test, y_pred))

0.7942521227955585


In [46]:
# Save
model.save('FakeNewsClassifier.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
